In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y

In [ ]:
import numpy as np
import pandas as pd
import os
import pydicom
from tqdm.notebook import tqdm

You can also use the outputs of this notebook as data to import instead of running the code yourself.<br>
Make sure to use .loc[ ] when indexing custom indexes :)<br>
Hope you find something useful, have fun!

In [ ]:
#Train image shapes and paths
images = []
widths = []
heights = []
paths = []

tq = tqdm()

for dirname, _, filenames in os.walk('../input/siim-covid19-detection/train'):
    for filename in filenames:
        dicom = pydicom.read_file(dirname+"/"+filename,specific_tags=["Rows","Columns"])
        heights.append(dicom.Rows)
        widths.append(dicom.Columns)
        images.append(filename[:-4])
        paths.append(dirname+"/"+filename)
        tq.update(1)

id2path_train = pd.DataFrame({"Image_id":images,"Path":paths})    
img_shapes_train = pd.DataFrame({"Image_id":images,"Width":widths,"Height":heights}).set_index("Image_id")
img_shapes_train

In [ ]:
id2path_train

In [ ]:
#Study to Image
train_image_level = pd.read_csv("../input/siim-covid19-detection/train_image_level.csv")
image_study_train = pd.DataFrame({"Image_id":train_image_level.id.str[:-6],"Study_id":train_image_level.StudyInstanceUID})
image_study_train

In [ ]:
#Numeric Label and removed _study
train_study_level = pd.read_csv("../input/siim-covid19-detection/train_study_level.csv")
train_study_level.id = train_study_level.id.str[:-6]
train_study_level["Label"] = train_study_level.apply(lambda l: np.argmax(l.values[1:]),axis=1)
train_study_level

In [ ]:
#Image id to Classes 
img_classes_train = pd.DataFrame({"Image_id":image_study_train.Image_id,"Label":train_study_level.set_index("id").Label.loc[image_study_train.Study_id].values})
img_classes_train

In [ ]:
#Helper function
def getBoxes(x):
    if not pd.isnull(x.boxes):
        new = []
        boxes = eval(x.boxes)
        width = img_shapes_train.loc[x.id[:-6]].Width
        height = img_shapes_train.loc[x.id[:-6]].Height
        for box in boxes:
            new.append({"x":max(0,box["x"]/width),
                        "y":max(0,box["y"]/height),
                        "width":max(0,box["width"]/width),
                        "height":max(0,box["height"]/height)})# There is one value below 0 --> max(0,value)
        return new
    else:
        return 

In [ ]:
#Normalized Boxes
#Use eval() to turn the string into a list
img_boxes_train =  pd.DataFrame({"Image_id":train_image_level.id.str[:-6], "Boxes":train_image_level.apply(getBoxes,axis=1),"has_box":~pd.isnull(train_image_level.boxes)})
img_boxes_train

In [ ]:
#Test set paths, shapes and image to study
studies = []
images = []
paths = []
widths = []
heights = []
tq = tqdm()

for dirname, _, filenames in os.walk('../input/siim-covid19-detection/test'):
    for filename in filenames:
        dicom = pydicom.read_file(dirname+"/"+filename,specific_tags=["Rows","Columns"])
        heights.append(dicom.Rows)
        widths.append(dicom.Columns)
        studies.append(dirname.split("/")[-2])
        images.append(filename[:-4])
        paths.append(dirname+"/"+filename)
        tq.update(1)

id2path_test = pd.DataFrame({"Image_id":images,"Path":paths})
img_study_test = pd.DataFrame({"Study_id":studies,"Image_id":images})    
img_shapes_test = pd.DataFrame({"Image_id":images,"Width":widths,"Height":heights}).set_index("Image_id")

In [ ]:
img_study_test

In [ ]:
img_shapes_test

In [ ]:
id2path_test

In [ ]:
#Output
id2path_train.to_csv("id2path_train.csv",index=False)
image_study_train.to_csv("image_study_train.csv",index=False)
img_shapes_train.to_csv("img_shapes_train.csv")

img_classes_train.to_csv("img_classes_train.csv",index=False)
img_boxes_train.to_csv("img_boxes_train.csv",index=False)

id2path_test.to_csv("id2path_test.csv",index=False)
img_study_test.to_csv("img_study_test.csv",index=False)
img_shapes_test.to_csv("img_shapes_test.csv")